In [10]:
import os
import openai
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Domain description
domain = (
    "In this domain, two agents representing two people living together negotiate in a local supermarket who have different tastes. The domain consists of 5 types of products (or issues): bread, fruit, snacks, spreads, and vegetables. Each category has 4 or 5 products, resulting in a medium-sized domain with 1600 possible outcomes."
)

def enrich_explanation(sentence):
    """
    Function to generate a concise explanation for a given LaTeX sentence.
    Args:
    - sentence (str): LaTeX mathematical expression to explain.

    Returns:
    - str: Concise explanation of the LaTeX expression.
    """
    # Refined prompt for brevity and simplicity
    prompt = (
        f"Given the domain context below, provide a very short and concise explanation of the mathematical expression, "
        f"focusing on the key conditions within each interval. Keep each explanation to just one or two sentences.\n\n"
        f"Domain Context:\n{domain}\n\n"
        f"Mathematical Expression: {sentence}\n\n"
        f"Explanation Format:\n"
        f"- In the first interval [...]: [very short explanation].\n"
        f"- In the second interval [...]: [very short explanation].\n\nExplanation:"
    )

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You're an expert assistant who provides very short, clear, and concise explanations of mathematical expressions in specific domains."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,  # Further reducing max_tokens to enforce brevity
        temperature=0.3,  # Lowering temperature for more straightforward output
        top_p=0.9,
        frequency_penalty=0.0,
        presence_penalty=0.0,
    )

    enriched_explanation = response['choices'][0]['message']['content'].strip()
    return enriched_explanation

# LaTeX mathematical sentence to be explained
sentence = r"t \in [0.000, 0.1739) \rightarrow U_u(\omega_t^o) \geq \max\left( u ,\bar{u_t} \right)"

# Generate concise explanation
enriched_sentence = enrich_explanation(sentence)
print(f"Enriched Explanation:\n{enriched_sentence}\n")


Enriched Explanation:
- In the first interval [0.000, 0.1739): The utility of the offer made by the other agent at time t must be greater than or equal to the maximum of the agent's utility and the average utility up to time t.



In [11]:
def prompt_layperson(enriched_sentence):
    return (
        "Your task is to explain the following mathematical statement in very simple terms, suitable for someone without any technical background. The explanation should be clear, concise, and within 30 words. Avoid using any jargon or complex terms. Refer to the examples below for the style of explanation:\n\n"
        f"**Mathematical Statement:**\n{enriched_sentence}\n\n"
        "**Examples of Clear Explanations for a Layperson:**\n"
        "1. The final price should match the average market price or include a discount, ensuring it is fair and competitive.\n"
        "2. In the first phase, the plan should improve basic features to be at least as good as a standard option.\n"
        "3. The service package should meet a basic quality level or reach a specific customer satisfaction score to ensure a good experience.\n"
        "4. The initial budget must be large enough to cover all estimated costs and any additional expenses.\n\n"
        "**Your Task:**\n"
        "Based on the mathematical statement provided, generate a clear and simple explanation suitable for a layperson, within 50 words."
    )


# Prompt for expert explanation


def prompt_expert(enriched_sentence):
    return (
        "Provide a detailed and technical explanation of the following mathematical statement for a domain expert. The explanation should be within 50 words. Refer to the examples below for the style of explanation:\n\n"
        f"**Mathematical Statement:**\n{enriched_sentence}\n\n"
        "**Explanation for Domain Expert:**\n"
        "1. During the second interval [0.0361, 1.000], the utility of the opponent's offer \( U_u(\omega_t^o) \) must exceed the higher of a predefined threshold \( u \) or the quantile function \( U_{\Omega^o_t} \) at a specific time-dependent point.\n"
        "2. The initial evaluation phase requires the service package value \( V_s \) to surpass the minimum quality benchmark or meet a defined satisfaction threshold to ensure compliance with service standards.\n"
        "3. The order quantity \( Q_s \) must align with the highest value between the minimum stock level and a demand forecast quantile to optimize inventory management during the initial stocking phase.\n\n"
        "**Your Task:**\n"
        "Provide a similar style explanation suitable for an expert, within 50 words."
    )

In [12]:
enriched_sentence = "In the first interval [0.000, 0.1739): The utility of the offer made by the other agent at time t must be greater than or equal to the maximum of the agent's utility and the average utility up to time t."


In [13]:
def custom_explanation(sentence, target_audience, prompt_func, confidence_score=None, max_tokens=400, temperature=0.6, top_p=0.7, frequency_penalty=0.0, presence_penalty=0.0):
    # Generate the initial prompt based on the target audience
    prompt = prompt_func(enriched_sentence)

    # If confidence_score is provided, generate feedback
    if confidence_score is not None:
        feedback = generate_feedback(
            enriched_sentence, confidence_score, target_audience)
        prompt += f"\n\nFeedback for Improvement:\n{feedback}\n\nRefine the explanation based on the feedback."
    else:
        feedback = None

    # Use OpenAI's API to get the explanation
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert assistant. Your task is to provide clear and concise explanations for the specified audience."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
    )

    # Extract the custom explanation from the response
    custom_explanation = response['choices'][0]['message']['content'].strip()

    # Check if the response is close to the token limit and add a note if it is
    if len(custom_explanation) >= max_tokens - 20:
        custom_explanation += " (response cut off, please refine or increase token limit)"

    return custom_explanation



In [14]:

# Example usage
layperson_output = custom_explanation(
    enriched_sentence, 'layperson', prompt_layperson)

expert_output = custom_explanation(enriched_sentence, 'expert', prompt_expert)

print("Layperson Explanation Output:\n", layperson_output)
print("\nExpert Explanation Output:\n", expert_output)

Layperson Explanation Output:
 In the beginning stage, the other person's offer should be at least as good as the best deal you've seen so far, or the average of all deals up to that point.

Expert Explanation Output:
 In the initial interval [0.000, 0.1739), the other agent's offer utility \( U_u(\omega_t^o) \) must be at least as high as the maximum of the agent's own utility or the mean utility up to time \( t \), ensuring optimal decision-making based on prevailing conditions and historical performance.


In [17]:
import os
import openai
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Domain description
domain = (
    "In this domain, two agents representing two people living together negotiate in a local supermarket who have different tastes. The domain consists of 5 types of products (or issues): bread, fruit, snacks, spreads, and vegetables. Each category has 4 or 5 products, resulting in a medium-sized domain with 1600 possible outcomes."
)

# Prompts for different audience types
def prompt_layperson(enriched_sentence):
    return (
        "Your task is to explain the following mathematical statement in very simple terms, suitable for someone without any technical background. "
        "The explanation should be clear, concise, and within 30 words. Avoid using any jargon or complex terms. Refer to the examples below for the style of explanation:\n\n"
        f"**Mathematical Statement:**\n{enriched_sentence}\n\n"
        "**Examples of Clear Explanations for a Layperson:**\n"
        "1. The final price should match the average market price or include a discount, ensuring it is fair and competitive.\n"
        "2. In the first phase, the plan should improve basic features to be at least as good as a standard option.\n"
        "3. The service package should meet a basic quality level or reach a specific customer satisfaction score to ensure a good experience.\n"
        "4. The initial budget must be large enough to cover all estimated costs and any additional expenses.\n\n"
        "**Your Task:**\n"
        "Based on the mathematical statement provided, generate a clear and simple explanation suitable for a layperson, within 50 words."
    )

def prompt_expert(enriched_sentence):
    return (
        "Provide a detailed and technical explanation of the following mathematical statement for a domain expert. "
        "The explanation should be within 50 words. Refer to the examples below for the style of explanation:\n\n"
        f"**Mathematical Statement:**\n{enriched_sentence}\n\n"
        "**Explanation for Domain Expert:**\n"
        "1. During the second interval [0.0361, 1.000], the utility of the opponent's offer \\( U_u(\\omega_t^o) \\) must exceed the higher of a predefined threshold \\( u \\) or the quantile function \\( U_{\\Omega^o_t} \\) at a specific time-dependent point.\n"
        "2. The initial evaluation phase requires the service package value \\( V_s \\) to surpass the minimum quality benchmark or meet a defined satisfaction threshold to ensure compliance with service standards.\n"
        "3. The order quantity \\( Q_s \\) must align with the highest value between the minimum stock level and a demand forecast quantile to optimize inventory management during the initial stocking phase.\n\n"
        "**Your Task:**\n"
        "Provide a similar style explanation suitable for an expert, within 50 words."
    )

# Function to generate unique explanations
def generate_unique_explanations(sentence, audience, prompt_func, max_tokens=100, temperature=0.6):
    """
    Generate 20 unique explanations for the given sentence and audience.
    
    Args:
    - sentence (str): The LaTeX mathematical expression to explain.
    - audience (str): Type of audience ('layperson' or 'expert').
    - prompt_func (function): The function to generate the prompt for OpenAI.
    - max_tokens (int): Maximum number of tokens for the OpenAI response.
    - temperature (float): Sampling temperature for OpenAI API.
    
    Returns:
    - list of str: A list of 20 unique explanations.
    """
    unique_explanations = set()
    
    while len(unique_explanations) < 7:
        prompt = prompt_func(sentence)
        
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": f"You are an expert assistant. Your task is to provide clear and concise explanations for the specified audience: {audience}."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=0.9,
            frequency_penalty=0.0,
            presence_penalty=0.0,
        )

        explanation = response['choices'][0]['message']['content'].strip()
        
        # Ensure explanations are unique
        if explanation not in unique_explanations:
            unique_explanations.add(explanation)
    
    return list(unique_explanations)

# LaTeX mathematical sentence to be explained
sentence = r"t \in [0.1739, 0.2104) \rightarrow &  \
U_u(\omega_t^o) \geq \max\left( Q_{U_{\Omega^o_t}} (-0.10 \cdot t + 0.17), \bar{u_t}\right)\\"

# Generate 20 unique explanations for both layperson and expert
layperson_explanations = generate_unique_explanations(sentence, 'layperson', prompt_layperson)
expert_explanations = generate_unique_explanations(sentence, 'expert', prompt_expert)

# Print the outputs
print("Layperson Explanations:\n", "\n".join(layperson_explanations))
print("\nExpert Explanations:\n", "\n".join(expert_explanations))


Layperson Explanations:
 If a value falls between 0.1739 and just below 0.2104, the outcome should be at least as good as the higher of two things: a decrease by 10% of the value plus 0.17, or a set minimum level.
When the value 't' falls between 0.1739 and 0.2104 (but not including 0.2104), the outcome 'U' should be equal to or greater than the highest of two measures: one that decreases as 't' increases, and a standard level 'u'.
When a certain value is between 0.1739 and 0.2104, but not including 0.2104, the output should be at least as high as the result of a certain calculation or a fixed value, whichever is higher.
When a certain value falls within a specific range, the outcome should either match or be greater than the highest of two different calculations, ensuring the best possible result.
If a number is between 0.1739 and just under 0.2104, then a certain value (U) must be at least as big as either a slightly adjusted version of this number or another specific value, whicheve

In [2]:
import pandas as pd

# Load the two datasets
df1 = pd.read_csv('modified_expert_explanation_dataset.csv')
df2 = pd.read_csv('modified_layperson_explanation_dataset.csv')

# Merge the two datasets
merged_df = pd.concat([df1, df2], ignore_index=True)

# Shuffle the merged dataset
shuffled_df = merged_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the shuffled dataset to a new CSV file
shuffled_df.to_csv('merged_shuffled_dataset.csv', index=False)

print("Datasets have been merged, shuffled, and saved as 'merged_shuffled_dataset.csv'.")


Datasets have been merged, shuffled, and saved as 'merged_shuffled_dataset.csv'.
